# 1. Load Model

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

# check GPU status
print("Is CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

# load pre-trained model InceptionV3 and set mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True).to(device)
model.eval()

Is CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3060 Ti


C:\Users\jiayang\anaconda3\envs\pytorch2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jiayang\anaconda3\envs\pytorch2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,  random_split

def split_data_set(dataset, random_seed):
    if random_seed is not None:
        torch.manual_seed(random_seed)  # set input as random seed

    # split image set ---> half for calibration data set, half for test data set
    dataset_length = len(dataset)
    print(f"Samples amount: {dataset_length}")
    calib_length = dataset_length // 2               
    test_length = dataset_length - calib_length      

    calib_dataset, test_dataset = random_split(dataset, [calib_length, test_length])
    return calib_dataset, test_dataset

In [12]:
import numpy as np

# conformal function s(x,y)
def conformal_scores(model, dataloader, alpha=0.1, lambda_reg=0.1, k_reg=5):
    scores = []  # conformal scores of image sets
    labels = []  # true label sets 
    with torch.no_grad():
        for images, true_labels in dataloader:
            images, true_labels = images.to(device), true_labels.to(device)
            # logistic value
            outputs = model(images)
            # logistic value -> softmax
            # dim=1 : convert logistic values for all the classes of the example to the softmax 
            softmaxs = torch.softmax(outputs, dim=1)
            
            for softmax, true_label in zip(softmaxs, true_labels):
                # descending sort softmax
                sorted_softmax, sorted_index = torch.sort(softmax, descending=True)
                
                # get the position of the true label in the sorted softmax
                true_label_position = (sorted_index == true_label).nonzero(as_tuple=True)[0].item()
                # independent random variable u ~ Uniform(0, 1)
                u = np.random.uniform(0, 1)
                # cumulate sorted softmax
                cumulative_softmax = torch.cumsum(sorted_softmax, dim=0)  # dim=0 -> cumulate by raw direction

                # calculate p_x(y) + pi_x(y)*u
                if true_label_position == 0:
                    # first softmax is true label -> p_x(y) = 0 ; pi_x(y) = sorted_softmax[0]
                    p_and_pi = u * sorted_softmax[true_label_position].item()
                else:
                    p_and_pi = cumulative_softmax[true_label_position - 1].item() + u * sorted_softmax[true_label_position].item()

                # calculate regularization term: lamba * ( o_x(y) - k_reg)+
                regularization_term = lambda_reg * max( true_label_position + 1 - k_reg, 0)
                conformal_score = p_and_pi + regularization_term
                    
                scores.append(conformal_score)
                labels.append(true_label.item())
    return np.array(scores), np.array(labels)

In [13]:
def raps_classification(model, dataloader, t_cal, lambda_reg=0.1, k_reg=5):
    raps = []         # probability set
    raps_labels = []  # label set indicated to the probability set
    labels = []       # true label
    with torch.no_grad():
        for images, true_labels in dataloader:
            images, true_labels = images.to(device), true_labels.to(device)
            outputs = model(images)
            softmaxs = torch.softmax(outputs, dim=1)

            # sort softmax probabilities
            sorted_softmax, sorted_indices = torch.sort(softmaxs, descending=True, dim=1)  # shape: [batch_size, 1000]
            cumulative_softmax = torch.cumsum(sorted_softmax, dim=1)                       # shape: [batch_size, 1000]

            # rank of current sorted probability: [1,2,3,...,1000]
            rank = torch.arange(1, sorted_softmax.size(1) + 1, device=device).unsqueeze(0)  # shape: [1, 1000]
            # calculate regularization term
            regularization_term = lambda_reg * torch.clamp(rank - k_reg, min=0)             # shape: [1, 1000]

            # generate random variable u for all samples
            u = torch.rand_like(sorted_softmax)  # shape: [batch_size, 1000]

            # E = cumulative[current-1] + u*sorted[current] + regularization
            # which is equal to: cumulative[current] - sorted[current] + u*sorted[current] + regularization
            e = cumulative_softmax - sorted_softmax + u * sorted_softmax + regularization_term  # shape: [batch_size, 1000]

            e_less_than_t = e <= t_cal  
            # the first index of e that exceeds t_cal
            # if all the e <= t_cal  -> return 0
            e_exceed_t = (e > t_cal).float().argmax(dim=1) 
            cutoff_indices = torch.maximum(torch.sum(e_less_than_t, dim=1), e_exceed_t)  

            # build prediction sets
            for i, cutoff_index in enumerate(cutoff_indices):
                cutoff_index = max(cutoff_index.item(), 1)  # ensure cutoff_index >= 1
                raps.append(sorted_softmax[i, :cutoff_index].tolist())  
                raps_labels.append(sorted_indices[i, :cutoff_index].tolist())  
                labels.append(true_labels[i].item())  
    return raps, raps_labels, labels

In [14]:
def eval_aps(aps_labels, true_labels):
    total_set_size = 0
    coveraged = 0
    for aps_label, true_label in zip(aps_labels, true_labels):
        # cumulate total set size
        total_set_size += len(aps_label)
        # cumulate the predictions sets if it contains true label
        if true_label in aps_label:
            coveraged += 1

    # calculate average values
    samples_amount = len(true_labels)
    average_set_size = total_set_size / samples_amount
    average_coverage = coveraged / samples_amount
    print(f"Total set size: {total_set_size}")
    print(f"Total coverage sets: {coveraged}")
    print(f"Total samples amount: {samples_amount}")
    return average_set_size, average_coverage

In [16]:
# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
lambda_reg = 0.1
k_reg = 5

#  Reprocess: Center Crop and then resize to 299*299
data_transform = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.Resize(256), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])


import matplotlib.pyplot as plt
from PIL import Image

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)


# contruct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("APS Classification, Start!\n")


for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    # calculate q_hat
    calib_scores, _ = conformal_scores(model, calib_loader, alpha, lambda_reg, k_reg)
    t_cal = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Rate in runs {i+1}: {100 * avg_coverage:.2f}%\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)

print(f"Final Average Prediction Set Size: {final_avg_set_size}")
print(f"Final Average Coverage: {final_avg_coverage}")

APS Classification, Start!

Running experiment 1/10...
Samples amount: 50000
t_cal = 0.9744938526163025
Total set size: 92533
Total coverage sets: 22681
Total samples amount: 25000
Average Prediction Set Size After APS in runs 1: 3.70132
Average Coverage Rate in runs 1: 90.72%

Running experiment 2/10...
Samples amount: 50000
t_cal = 0.9738277707855308
Total set size: 91940
Total coverage sets: 22639
Total samples amount: 25000
Average Prediction Set Size After APS in runs 2: 3.6776
Average Coverage Rate in runs 2: 90.56%

Running experiment 3/10...
Samples amount: 50000
t_cal = 0.9759597020069737
Total set size: 93128
Total coverage sets: 22704
Total samples amount: 25000
Average Prediction Set Size After APS in runs 3: 3.72512
Average Coverage Rate in runs 3: 90.82%

Running experiment 4/10...
Samples amount: 50000
t_cal = 0.9730953148884834
Total set size: 91968
Total coverage sets: 22618
Total samples amount: 25000
Average Prediction Set Size After APS in runs 4: 3.67872
Average Co

## Result
From the above test, following results can be collected :
- Final Average Prediction Set Size: 3.6 / 1000
- Final Average Coverage: 90.55% ($\alpha$=0.1)